In [ ]:
!pip install faiss-cpu groq

In [ ]:
# run_query.py
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
# Function to embed queries
def embed_query(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return embeddings

In [ ]:
# Load embeddings from disk
embedding_matrix = np.load('embeddings.npy')
print("Embedding matrix shape:", embedding_matrix.shape)

# Load chunks from disk
with open('chunks.npy', 'rb') as f:
    splits = np.load(f, allow_pickle=True)
print("Splits loaded from disk.")

Embedding matrix shape: (331, 1024)
Splits loaded from disk.


In [ ]:
# Define the directory where the model and tokenizer are saved
load_directory = "./saved_embedding_model"

# Load the tokenizer from the same directory
tokenizer = AutoTokenizer.from_pretrained(load_directory, trust_remote_code=True)

# Load the model from the same directory
model = AutoModel.from_pretrained(load_directory, trust_remote_code=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NewModel(
  (embeddings): NewEmbeddings(
    (word_embeddings): Embedding(30528, 1024, padding_idx=0)
    (rotary_emb): NTKScalingRotaryEmbedding()
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): NewEncoder(
    (layer): ModuleList(
      (0-23): 24 x NewLayer(
        (attention): NewSdpaAttention(
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): NewGatedMLP(
          (up_gate_proj): Linear(in_features=1024, out_features=8192, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (act_fn): GELUActivation()
          (hidden_dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-12, elementwis

In [ ]:
# Load the Faiss index
index = faiss.read_index('faiss_index.index')
print("Faiss index loaded from disk.")

Faiss index loaded from disk.


In [ ]:
# Function to search the index
def search_index(query, tokenizer, model, index, splits, top_k=5):
    query_embedding = embed_query(query, tokenizer, model)
    query_embedding = np.expand_dims(query_embedding, axis=0).astype('float32')  # Faiss expects a 2D array of float32
    distances, indices = index.search(query_embedding, top_k)
    return [splits[idx] for idx in indices[0]]

In [ ]:
# Example query
query = "What is the procedure for opening minor account?"
results = search_index(query, tokenizer, model, index, splits)

# Join the results into a single string
context = "\n".join(results)

# Create the formatted string
formatted_string = f"Question: {query}\n\nContext: {context}"

# Print the formatted string
#print(formatted_string)

In [ ]:
import os
os.environ['GROQ_API_KEY'] = ' ' #Groq API Key

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": formatted_string,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

To open a minor account, the following procedure should be followed:

1. Remarks "Minor Account" should be mentioned on the account opening form and signature specimen card, and on the system during uploading specimen signature.
2. Based on the date of birth mentioned on the birth certificate, the date when the minor becomes major must be clearly noted on both the account opening form and specimen signature card as "Minority concludes on…………… (dd/mm/yy)".
3. The account should be opened and operated by the guardian/legal guardian till the minor becomes major.
4. A passport size photo of the account holder (minor) is required.
5. On majority, the account holder is entitled to operate the account as a regular account instead of a minor account. A written request of the account holder and guardian should be obtained to change the mode of account operation/account closure.
6. Identification documents along with a new account opening form of the account holder should be obtained and updated